# Исследование надежности заемщиков.

Описание проекта.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

In [28]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Описание данных

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Изучение пропусков

In [30]:
display(data.isna().sum())
print('Доля пропущенных значений в столбце days_employed: {:.0%}'.format(data['days_employed'].isna().sum()/data['days_employed'].count()))
print('Доля пропущенных значений в столбце total_income: {:.0%}'.format(data['total_income'].isna().sum()/data['total_income'].count()))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Доля пропущенных значений в столбце days_employed: 11%
Доля пропущенных значений в столбце total_income: 11%


В столбцах days_employed и total_income пропушены 2174 значения. Пропущенные значения являются колличественными, так как принимают любое числовое значение в диапазоне. Доля пропущенных значений от общего кол-ва значений столбцов составляет 11% в двух случаях. 

Причины появления пропусков могут быть технологические (из-за копирования, считывания или переформатирования) или из-за человеского фактора.

Лучше использовать медиану, чтобы в случае присутствия выдающихся значений, среднее значение данных не искажалось.

In [31]:
days_employed_avg = data['days_employed'].median()
total_income_avg = data['total_income'].median()
print(days_employed_avg)
print(total_income_avg)

-1203.369528770489
145017.93753253992


Явная ошибка в значение days_employed_avg, средний стаж работы больше средних лет жизни человека. Где-то скорее всего есть аномальные значения, которые влияют на среднее значение.
Ошибка в отрицательном значении среднего дохода total_income_avg.


In [32]:
data['days_employed'] = data['days_employed'].fillna(days_employed_avg)
data['total_income'] = data['total_income'].fillna(total_income_avg)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [33]:
display(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропущенных значений нет.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В первую очередь приведем значения к положительным, чтобы далее было удобно работать с данными.
Аномалии могут возникнуть из-за ошибки в форматировании, при скачивании файла или из-за человеского фактора.


In [34]:
data['children'] = abs(data['children'])
data['days_employed'] = abs(data['days_employed'])
data['dob_years'] = abs(data['dob_years'])
data['total_income'] = abs(data['total_income'])
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [35]:
max_days_employed = data['days_employed'] // 365 > data['dob_years'] - 18
data.loc[max_days_employed, 'days_employed'] = (data.loc[max_days_employed,'dob_years']-18)*365
display(data.head(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6205.000000,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Для изменения данных по стажу посчитаем возможен ли стаж каждого человека в зависимости от его возраста. Примем за начало работы возраст 18 лет. Используем фильтрацию для замены аномальных значений.

### Шаг 2.3. Изменение типов данных.

In [36]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6205,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Применяем функцию int() для того, чтобы изменить float на int. 

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Данные float теперь целочисленные.

### Шаг 2.4. Удаление дубликатов.

In [38]:
print(data['family_status'].unique())
print(data['education'].unique())
print(data['income_type'].unique())
print(data['purpose'].unique())
print(data['gender'].unique())
print(data['debt'].unique())
print(data['children'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка

В education разный регистр букв, что создает неявные дубликаты.
Также, заметно, что цели в purpose похожи, но написаны разными словами.
Также в столбце gender есть неизвестный пол XNA.
В столбце children есть показатель в 20 детей, что странно. Возьмем 20 за 2, так как возможна опечатка.

In [39]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Устранили неявные дубликаты.
Выбран метод unique, чтобы посмотреть уникальные значения столбца и сравнить их.

In [40]:
data['gender'] = data['gender'].replace('XNA', 'F')
print(data['gender'].unique())

['F' 'M']


Заменили XNA на женский пол, теперь у нас только женский и мужской пол в столбце.

In [41]:
data['children'] = data['children'].replace(20, 2)
print(data['children'].unique())

[1 0 3 2 4 5]


Заменили аномальное кол-во детей.

In [42]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

0


Удалены явные дубликаты, чтобы не было искажения финальных оценок.

Похоже, что строки просто задвоились, что привело к дубликатам.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [43]:
data_education = data[['education_id', 'education']].drop_duplicates().set_index('education_id')
data_family = data[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')

display(data_education)
display(data_family)

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Созданы две таблицы, это будут наши словари, к которым можно будет обращаться по  id.

In [44]:
data.pop('education')
data.pop('family_status')
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6205,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


Удалили два столбца, которые теперь находятся отдельно в словарях. Таблица стала компактней и удобней. 

### Шаг 2.6. Категоризация дохода.

In [45]:
def salary_type(total_income):

    if total_income >= 0 and total_income <= 30000:
        return 'E'
    elif total_income >= 30000 and total_income <= 50000:
        return 'D'
    elif total_income >= 50001 and total_income <= 200000:
        return 'C'
    elif total_income >= 200001 and total_income <= 1000000:
        return 'B'
    else:
        return 'A'

data['total_income_category'] = data['total_income'].apply(salary_type)
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6205,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Cоздан столбец total_income_category с категориями, которые распределяются исходя из total_income. 
Использовали конструкцию try-except, чтобы в случае с неподходящими ни под одну категорию данными, код не упал.


### Шаг 2.7. Категоризация целей кредита.

In [46]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Изучили повторяющиеся цели, далее объединим их в категории.

In [47]:
def purpose_f (purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'недвижим'in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'образова' in purpose:
        return 'получение образования'
    else:
        return 'операции с автомобилем'

data['purpose_category'] = data['purpose'].apply(purpose_f)
display(data.head(10))  

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6205,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Создана функция, которая добавляет в новый столбец категорию, характеризующую цель взятия кредита.
Использовали конструкцию try-except, чтобы в случае с неподходящими ни под одну категорию данными, код не упал.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [48]:
data.groupby('children').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean'))\
    .sort_values('доля_должников', ascending=False)\
    .style.format({'доля_должников' : '{:.2%}'.format})


,число_клиентов,число_должников,доля_должников
children,,,
4,41,4,9.76%
2,2128,202,9.49%
1,4855,445,9.17%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


##### Вывод 1: Заемщики, у которых нет детей, менее склонны к просрочке по выплатам кредита. Чем больше детей - тем больше возможность задолженности.

0.9.0.2  Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [49]:
data.groupby('family_status_id').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean'))\
    .sort_values('доля_должников', ascending=False).join(data_family)\
    .style.format({'доля_должников' : '{:.2%}'.format})

,число_клиентов,число_должников,доля_должников,family_status
family_status_id,,,,
4,2810,274,9.75%,Не женат / не замужем
1,4151,388,9.35%,гражданский брак
0,12339,931,7.55%,женат / замужем
3,1195,85,7.11%,в разводе
2,959,63,6.57%,вдовец / вдова


##### Вывод 2: Заемщики, которые состоят в браке или были в браке, менее склонны к просрочке по выплатам кредита. Заемщики, которые не были в узаконенных отношениях чаще имеют долг по кредиту.

0.9.0.3  Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
data_pivot_income = data.pivot_table(index = data['total_income_category'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
data_pivot_income['ratio'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1])
data_pivot_income.sort_values('ratio', ascending = False)

debt,0,1,ratio
total_income_category,,,
E,20,2,0.090909
C,14656,1360,0.084915
A,23,2,0.080000
B,4685,356,0.070621
D,329,21,0.060000


##### Вывод 3: Заемщики с наименьшим уровнем дохода  более склонны нарушать обязательства по выплатам кредита в срок. Но заемщики среднего уровня дохода лучше всех выполняют свои обязательства.

0.9.0.4 Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [51]:
data_pivot_purpose = data.pivot_table(index = data['purpose_category'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
data_pivot_purpose['ratio'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1])
data_pivot_purpose.sort_values('ratio', ascending = False)

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,0.093590
получение образования,3643,370,0.092200
проведение свадьбы,2138,186,0.080034
операции с недвижимостью,10029,782,0.072334


Заемщики, которые берут кредит для проведения операций с жильем менее склонны нарушать обязательства по выплатам кредита в срок. Заемщики, которые берут кредит на автомобиль наоборот чаще задерживают выплаты.

## Общий вывод:

Цель анализа – узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Исходя из данных, можно сделать вывод, что кол-во детей влияет на заемщиков. Чем больше детей в семье – тем чаще люди задерживают выплаты.
Относительно семейного положения, можно сказать, что люди, которые не были в браке чаще становятся должниками.

Итог:
заемщики без детей и в браке (или были когда-то в браке) – самые ответственные заемщики.
Заемщики с детьми и не находящиеся в браке – менее отвественные заемщики.